<a href="https://colab.research.google.com/github/jyulkm/Tyler-Tech-Practicum/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import pandas as pd
from openai import OpenAI
import requests
import torch
import re


In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00


In [3]:
data = pd.read_csv('HHS.csv', usecols=['Name', 'Description', 'Tags'])

In [4]:
data['name_description'] = data['Name'] + ' ' + data['Description']

In [5]:
data

,Name,Description,Tags,name_description
0,HAICViz - MuGSI,<p>The healthcare-associated infection compone...,"antimicrobial resistance,carbapenem-resistant ...",HAICViz - MuGSI <p>The healthcare-associated i...
1,HHS Data Governance Board Membership - w3i2-ja...,This dataset tracks the updates made on the da...,NaN,HHS Data Governance Board Membership - w3i2-ja...
2,Drug Interaction API - 5pe7-4rei - Archive Rep...,This dataset tracks the updates made on the da...,NaN,Drug Interaction API - 5pe7-4rei - Archive Rep...
3,NCHS Research and Development Survey (RANDS) R...,The Research and Development Survey (RANDS) is...,"gender identity,intimate partner violence,phys...",NCHS Research and Development Survey (RANDS) R...
4,"NNDSS - TABLE 1AA. Poliovirus infection, nonp...",This dataset tracks the updates made on the da...,NaN,"NNDSS - TABLE 1AA. Poliovirus infection, nonp..."
...,...,...,...,...
14454,HHS IDs,This file helps define the HHS_ID column that ...,"covid-19,hhs covid-19",HHS IDs This file helps define the HHS_ID colu...
14455,COVID-19 Nursing Home Dataset - ywg6-hgzb - Ar...,This dataset tracks the updates made on the da...,NaN,COVID-19 Nursing Home Dataset - ywg6-hgzb - Ar...
14456,Blog | The Health Consequences of Smoking—50 Y...,This blog post was posted by Boris Lushniak on...,blog,Blog | The Health Consequences of Smoking—50 Y...
14457,"Nutrition, Physical Activity, and Obesity - Be...","This dataset includes data on adult's diet, ph...","adults,brfss,dnpao,fruit,nutrition,obesity,ove...","Nutrition, Physical Activity, and Obesity - Be..."


In [ ]:
data['name_description'].iloc[0]

'HAICViz - MuGSI <p>The healthcare-associated infection component of CDC’s <a href="https://www.cdc.gov/ncezid/dpei/eip/index.html" target="_blank">EIP</a> engages a <a href="https://www.cdc.gov/ncezid/dpei/eip/eip-sites.html" target="_blank">network</a> of state health departments and their academic medical center partners to help answer critical questions about emerging HAI threats, advanced infection tracking methods, and antibiotic resistance in the United States. Information gathered through this activity will play a key role in shaping future policies and recommendations targeting HAI prevention. </p>'

In [50]:
data['openAI_tags'] = None

In [ ]:
client = OpenAI(api_key="sk-proj-OW3BIPRXsb4uhiwB1vO9T3BlbkFJNBekzWtxf16Wl2qBHmVS")

for i in range(data.shape[0]):
  completion = client.chat.completions.create(
    model="gpt-4-turbo-2024-04-09",
    messages=[
      {"role": "system", "content": "You are a tag generating assistant, skilled in extracting key information based on description."},
      {"role": "user", "content": f"Generate 5 tags in a single word based on description: {data['name_description'].iloc[i]}"}
    ]
  )

  print('-------------------------------------------------')
  print(data['name_description'].iloc[i])
  data['openAI_tags'][i] = completion.choices[0].message.content.replace('\n', ' ')
  print(completion.choices[0].message.content.replace('\n', ' '))

-------------------------------------------------
HAICViz - MuGSI <p>The healthcare-associated infection component of CDC’s <a href="https://www.cdc.gov/ncezid/dpei/eip/index.html" target="_blank">EIP</a> engages a <a href="https://www.cdc.gov/ncezid/dpei/eip/eip-sites.html" target="_blank">network</a> of state health departments and their academic medical center partners to help answer critical questions about emerging HAI threats, advanced infection tracking methods, and antibiotic resistance in the United States. Information gathered through this activity will play a key role in shaping future policies and recommendations targeting HAI prevention. </p>
1. Healthcare 2. CDC 3. Infection 4. AntibioticResistance 5. Policy
-------------------------------------------------
HHS Data Governance Board Membership - w3i2-jayx - Archive Repository This dataset tracks the updates made on the dataset "HHS Data Governance Board Membership" as a repository for previous versions of the data and met

In [36]:
completion.choices[0].message.content.replace('\n', ' ')

'1. Healthcare 2. Infection 3. Antibiotic 4. Prevention 5. Policy'

In [40]:
test = re.sub(r'\d+\.\s*', '', completion.choices[0].message.content.replace('\n', ' '))


'H, e, a, l, t, h, c, a, r, e,  , I, n, f, e, c, t, i, o, n,  , A, n, t, i, b, i, o, t, i, c,  , P, r, e, v, e, n, t, i, o, n,  , P, o, l, i, c, y'

## KeyBert

In [6]:
pip install keybert

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 7.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cache

In [18]:
from keybert import KeyBERT

doc = data['name_description'].iloc[0]
kw_model = KeyBERT()

data['keybert_tags'] = data['name_description'].apply(lambda x: kw_model.extract_keywords(x))

#keywords = kw_model.extract_keywords(doc)

ValueError: Cannot set a DataFrame with multiple columns to the single column keybert_tags

In [ ]:
doc

'HAICViz - MuGSI <p>The healthcare-associated infection component of CDC’s <a href="https://www.cdc.gov/ncezid/dpei/eip/index.html" target="_blank">EIP</a> engages a <a href="https://www.cdc.gov/ncezid/dpei/eip/eip-sites.html" target="_blank">network</a> of state health departments and their academic medical center partners to help answer critical questions about emerging HAI threats, advanced infection tracking methods, and antibiotic resistance in the United States. Information gathered through this activity will play a key role in shaping future policies and recommendations targeting HAI prevention. </p>'

In [8]:
keywords

[('antibiotic', 0.4691),
 ('cdc', 0.4347),
 ('infection', 0.4223),
 ('prevention', 0.3929),
 ('haicviz', 0.3471)]

In [24]:
data['name_description'].apply(lambda x: kw_model.extract_keywords(x) if isinstance(x, str) else 'None')


TypeError: 'float' object is not iterable

In [23]:
data['name_description'].astype(str)

0        HAICViz - MuGSI <p>The healthcare-associated i...
1        HHS Data Governance Board Membership - w3i2-ja...
2        Drug Interaction API - 5pe7-4rei - Archive Rep...
3        NCHS Research and Development Survey (RANDS) R...
4        NNDSS - TABLE 1AA.  Poliovirus infection, nonp...
                               ...                        
14454    HHS IDs This file helps define the HHS_ID colu...
14455    COVID-19 Nursing Home Dataset - ywg6-hgzb - Ar...
14456    Blog | The Health Consequences of Smoking—50 Y...
14457    Nutrition, Physical Activity, and Obesity - Be...
14458    NNDSS - TABLE 1Y. Mumps to Novel influenza A v...
Name: name_description, Length: 14459, dtype: object

In [ ]:
kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 1), stop_words=None)

[('antibiotic', 0.4691),
 ('cdc', 0.4347),
 ('infection', 0.4223),
 ('prevention', 0.3929),
 ('haicviz', 0.3471)]

In [ ]:
kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 2), stop_words=None)

[('advanced infection', 0.5157),
 ('cdc gov', 0.5096),
 ('infection tracking', 0.509),
 ('hai prevention', 0.5069),
 ('www cdc', 0.4815)]

## Bart

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartForSequenceClassification

In [ ]:
# Load pretrained BART model and tokenizer
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# Description input
description = data['Description'][0]

# Tokenize and preprocess the description
input_ids = tokenizer(description, return_tensors="pt", max_length=1024, truncation=True)["input_ids"]

# Generate tags
output = model.generate(input_ids, max_length=50, num_beams=5, early_stopping=True)
generated_tags = tokenizer.decode(output[0], skip_special_tokens=True)

# Postprocess tags if needed
# For example, split the generated tags into a list
generated_tags_list = generated_tags.split(',')

# Display or use generated tags
print("Generated Tags:", generated_tags_list)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1202: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (50). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


Generated Tags: ['The healthcare-associated infection component of CDC’s EIP engages a network of state health departments and their academic medical center partners. Information gathered through this activity will play a key role in shaping future policies and recommendations targeting HAI prevention']


In [ ]:
# Load pretrained BART model and tokenizer for sequence classification
model_name = "facebook/bart-large-mnli"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForSequenceClassification.from_pretrained(model_name)

# Define text input for classification
text = data['Description'][0]

# Tokenize and preprocess the text
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

# Perform inference
with torch.no_grad():
    outputs = model(**inputs)

# Get predicted label
predicted_label_id = torch.argmax(outputs.logits, dim=1).item()

# Map label id to label name (if available)
label_map = {0: "entailment", 1: "neutral", 2: "contradiction"}  # Example mapping for MNLI task
predicted_label_name = label_map[predicted_label_id]

# Print predicted label
print("Predicted Label:", predicted_label_name)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Predicted Label: neutral


In [ ]:
data['Description'][0]

'<p>The healthcare-associated infection component of CDC’s <a href="https://www.cdc.gov/ncezid/dpei/eip/index.html" target="_blank">EIP</a> engages a <a href="https://www.cdc.gov/ncezid/dpei/eip/eip-sites.html" target="_blank">network</a> of state health departments and their academic medical center partners to help answer critical questions about emerging HAI threats, advanced infection tracking methods, and antibiotic resistance in the United States. Information gathered through this activity will play a key role in shaping future policies and recommendations targeting HAI prevention. </p>'

In [ ]:
generated_tags

'The healthcare-associated infection component of CDC’s EIP engages a network of state health departments and their academic medical center partners. Information gathered through this activity will play a key role in shaping future policies and recommendations targeting HAI prevention'